In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [3]:
df= pd.read_parquet("yellow_tripdata_2023-01.parquet")

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [5]:
len(df.columns)

19

In [6]:
df['duration']=df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']
df['duration']=df['duration'].dt.total_seconds().div(60).astype(int)
df['duration'].std()

42.59449915794647

In [7]:
df2 = df[(df['duration']>=1) & (df['duration']<=60)]
len(df2)/len(df)

0.9818303711466737

In [8]:
categorical = ['PULocationID', 'DOLocationID']
df2[categorical] = df2[categorical].astype(str)

/tmp/ipykernel_16920/3626608506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[categorical] = df2[categorical].astype(str)


In [9]:
train_dicts = df2[categorical].to_dict(orient='records')
dv = DictVectorizer()

In [10]:
X_train = dv.fit_transform(train_dicts)

In [11]:
X_train.shape[1]

515

In [12]:
target = 'duration'
y_train = df2[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.68865165928185

In [13]:
df3= pd.read_parquet("yellow_tripdata_2023-02.parquet")

In [14]:
df3['duration']=df3['tpep_dropoff_datetime']-df3['tpep_pickup_datetime']
df3['duration']=df3['duration'].dt.total_seconds().div(60).astype(int)

df4 = df3[(df3['duration']>=1) & (df3['duration']<=60)]

df4[categorical] = df4[categorical].astype(str)

val_dicts = df4[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = df4[target].values

/tmp/ipykernel_16920/145544189.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4[categorical] = df4[categorical].astype(str)


In [15]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.86898517326125